This notebook requires:
* the North Sea test case from the [getm-setups repository](https://github.com/getm-model/setups)
* the [TPXO9 atlas](https://www.tpxo.net/global/tpxo9-atlas) to calculate tides at the open boundaries

In [ ]:
getm_setups_dir = '../../../getm-setups'
igotm_data_dirs = ('/server/data', '../../../igotm/data')

In [ ]:
import datetime
import os.path

import pygetm
import pygetm.domain
import pygetm.input
import pygetm.legacy
import pygetm.input.tpxo
import pygetm.airsea

# Note: "%matplotlib widget" below enables interactive plots but requires https://github.com/matplotlib/ipympl
# Alternatively you could use "%matplotlib notebook" (deprecated).
%matplotlib widget
import matplotlib.pyplot

In [ ]:
domain = pygetm.legacy.domain_from_topo(os.path.join(getm_setups_dir, 'NorthSea/Topo/NS6nm.v01.nc'), nlev=1, z0_const=0.001)
pygetm.legacy.load_bdyinfo(domain, os.path.join(getm_setups_dir, 'NorthSea/bdyinfo.dat'))
sim = pygetm.Simulation(domain, runtype=1, apply_bottom_friction=True, advection_scheme=1)
print('Maximum dt = %s' % domain.maxdt)

fig, ax = matplotlib.pyplot.subplots()
ax.pcolormesh(domain.T.loni, domain.T.lati, domain.T.H.ma);

In [ ]:
fig, ax = matplotlib.pyplot.subplots(figsize=(12,12))
pc = ax.pcolormesh(domain.lon, domain.lat, domain.mask, shading='auto');
fig.colorbar(pc);
ax.plot(domain.T.lon[domain.bdy_j, domain.bdy_i], domain.T.lat[domain.bdy_j, domain.bdy_i], 'ow', mec='k', ms=2);
ax.set_title('mask on supergrid');

In [ ]:
igotm_data_dir = next(filter(os.path.isdir, igotm_data_dirs))

# Tidal boundary forcing from TPXO
tpxo_dir = os.path.join(igotm_data_dir, 'TPXO9')
bdy_lon = domain.T.lon[domain.bdy_j, domain.bdy_i]
bdy_lat = domain.T.lat[domain.bdy_j, domain.bdy_i]
tidal_h = pygetm.input.tpxo.get(bdy_lon, bdy_lat, root=tpxo_dir)
tidal_u = pygetm.input.tpxo.get(bdy_lon, bdy_lat, variable='u', root=tpxo_dir)
tidal_v = pygetm.input.tpxo.get(bdy_lon, bdy_lat, variable='v', root=tpxo_dir)

# Meteorology from ERA
era_path = os.path.join(igotm_data_dir, 'ERA-interim/2016.nc')
tcc = domain.T.map(pygetm.input.get_from_nc(era_path, 'tcc', preprocess=lambda ds: ds.isel(time=slice(4, -4))))
t2m = domain.T.map(pygetm.input.get_from_nc(era_path, 't2m', preprocess=lambda ds: ds.isel(time=slice(4, -4))))
d2m = domain.T.map(pygetm.input.get_from_nc(era_path, 'd2m', preprocess=lambda ds: ds.isel(time=slice(4, -4))))
sp = domain.T.map(pygetm.input.get_from_nc(era_path, 'sp', preprocess=lambda ds: ds.isel(time=slice(4, -4))))
u10 = domain.T.map(pygetm.input.get_from_nc(era_path, 'u10', preprocess=lambda ds: ds.isel(time=slice(4, -4))))
v10 = domain.T.map(pygetm.input.get_from_nc(era_path, 'v10', preprocess=lambda ds: ds.isel(time=slice(4, -4))))

In [ ]:
# Set up initial plot of elevation (to be animated later)
ref_date = datetime.datetime(2016, 1, 1)
fig, ax = matplotlib.pyplot.subplots()
pc = ax.pcolormesh(domain.T.loni, domain.T.lati, domain.T.z.ma, vmin=-2.5, vmax=2.5)
tidal_h.getm.update(ref_date)
sc = ax.scatter(bdy_lon, bdy_lat, c=tidal_h, edgecolors='k', vmin=-2.5, vmax=2.5, s=2, linewidth=.2)
cb = fig.colorbar(sc)
cb.set_label('elevation (m)')
title = ax.set_title(ref_date.strftime('%Y-%m-%d %H:%M'))

In [ ]:
inputs = t2m, d2m, sp, u10, v10, tidal_h.getm, tidal_u.getm, tidal_v.getm

timestep = 60.
timedelta = datetime.timedelta(seconds=timestep)
date = ref_date
for itime in range(60 *24 * 30):
    for variable in inputs:
        variable.update(date)

    # Update tidal elevations and velocities at the boundary
    sim.zbdy[:] = tidal_h
    sim.bdyu[:] = tidal_u
    sim.bdyv[:] = tidal_v

    es, ea, qs, qa, rhoa = pygetm.airsea.humidity(domain.T, 3, d2m - 273.15, sp, t2m - 273.15, t2m - 273.15)
    tausx, tausy, qe, qh = pygetm.airsea.airsea_fluxes(domain.T, 1, t2m, t2m, u10, v10, rhoa, qs, qa)
    sim.update_sealevel_boundaries(timestep)

    sim.update_surface_pressure_gradient(domain.T.z, sp)
    sim.uv_momentum_2d(timestep, tausx, tausy, sim.dpdx, sim.dpdy)
    sim.U.update_halos()
    sim.V.update_halos()
    sim.update_sealevel(timestep, sim.U, sim.V)
    sim.update_depth()

    date += timedelta

    if itime % 15 == 0:
        # Update plot
        title.set_text(date.strftime('%Y-%m-%d %H:%M'))
        sc.set_array(sim.zbdy)
        pc.set_array(domain.T.z.ma.ravel())

        fig.canvas.draw()